In [1]:
import os
os.chdir('../')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="7"

Fri May 19 16:25:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:1B:00.0 Off |                    0 |
| 30%   27C    P8    14W / 230W |   1433MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    Off  | 00000000:1C:00.0 Off |                  Off |
| 30%   

### Hyperparams

In [3]:
n_mels = 8
n_outputs = 61
n_frames = 400
sr = 24000
fps = 30
train_csv_files = ['/Storage/speech/face/rvh_visme_1_iPhone_30fps.csv',
                  '/Storage/speech/face/rvh_visme_2_iPhone_30fps.csv',
                  '/Storage/speech/face/rvh_visme_3_iPhone_30fps.csv']
train_wav_files = ['/Storage/speech/face/rvh_visme_1_iPhone.wav',
                 '/Storage/speech/face/rvh_visme_2_iPhone.wav',
                 '/Storage/speech/face/rvh_visme_3_iPhone.wav']

test_csv_files = ['/Storage/speech/face/rvh_visme_4_iPhone_30fps.csv']
test_wav_files = ['/Storage/speech/face/rvh_visme_4_iPhone.wav']

### Model

In [4]:
from model.model_vqvae import Model
from utils.util import *
from tensorboardX import SummaryWriter

step = 0
device = 'cuda:0'

# Model
model = Model(in_dim=n_mels, out_dim=n_outputs, K=32, latent_dim=256)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
print('done')

done


### Load

In [5]:
save_dir = '/data/scpark/save/lips/train05.19-1/'
!mkdir -p $save_dir
!ls -lt $save_dir

writer = SummaryWriter(save_dir)

if False:
    step, model, _, optimizer = load(save_dir, 1000, model, None, optimizer)

total 359380
-rw-rw-r-- 1 scpark scpark       131  5월 19 16:13 events.out.tfevents.1684480287.GPUSVR01
-rw-rw-r-- 1 scpark scpark       131  5월 19 16:11 events.out.tfevents.1684480187.GPUSVR01
-rw-rw-r-- 1 scpark scpark 367985211  5월 19 16:11 save_0
-rw-rw-r-- 1 scpark scpark       131  5월 19 16:09 events.out.tfevents.1684480037.GPUSVR01
-rw-rw-r-- 1 scpark scpark         0  5월 19 16:05 events.out.tfevents.1684479910.GPUSVR01
-rw-rw-r-- 1 scpark scpark       131  5월 19 16:04 events.out.tfevents.1684479520.GPUSVR01


In [6]:
import torch
from data.dataset import LipsDataset, Collate

dataset = LipsDataset(train_wav_files, train_csv_files, n_frames, n_mels=n_mels, sr=sr, fps=fps)
print('train dataset :', len(dataset))
train_loader = torch.utils.data.DataLoader(dataset, num_workers=1, shuffle=True, batch_size=8, 
                                           collate_fn=Collate(n_frames, n_mels))
print(train_loader)

dataset = LipsDataset(test_wav_files, test_csv_files, n_frames, n_mels=n_mels, sr=sr, fps=fps)
print('test dataset :', len(dataset))
test_loader = torch.utils.data.DataLoader(dataset, num_workers=1, shuffle=True, batch_size=8,
                                           collate_fn=Collate(n_frames, n_mels))
print(test_loader)

train dataset : 22042
test dataset : 7725


In [ ]:
from IPython import display
import librosa.display
import matplotlib.pyplot as plt

isnan = False
while True:
    if isnan:
        break
    for batch in train_loader:
        inputs = batch['inputs'].transpose(1, 2).to(device)
        targets = batch['outputs'].transpose(1, 2).to(device)
        
        model.train()
        model.zero_grad()
        outputs = model(inputs, targets)
        
        print(step)
        loss = 0
        for key in outputs.keys():
            if 'loss' in key:
                loss += outputs[key]
                print(key, outputs[key].item())
        if torch.isnan(loss):
            isnan = True
            break
        loss.backward()
        optimizer.step()
        print(step, loss.item())
        
        if step % 100 == 0:
            writer.add_scalar('train_loss', loss.item(), step)
        
        if step % 100 == 0:
            display.clear_output()
            
            losses = []
            for i, batch in enumerate(test_loader):
                if i >= 10:
                    break
                    
                inputs = batch['inputs'].transpose(1, 2).to(device)
                targets = batch['outputs'].transpose(1, 2).to(device)

                model.eval()
                with torch.no_grad():
                    outputs = model(inputs, targets)
                    
                loss = 0
                for key in outputs.keys():
                    if 'loss' in key:
                        loss += outputs[key]
                print('test :', i, loss.item())
                losses.append(loss)        
            
            test_loss = torch.stack(losses).mean().item()
            print('test_loss :', test_loss)
            writer.add_scalar('test_loss', test_loss, step)

        if step % 1000 == 0:
            save(save_dir, step, model, None, optimizer)
    
        step += 1

test : 0 2.051638126373291
test : 1 2.0733253955841064
test : 2 2.1308157444000244
test : 3 2.011448621749878
test : 4 1.9321786165237427
test : 5 2.1083908081054688
test : 6 1.9960181713104248
test : 7 2.1140997409820557
test : 8 2.1661970615386963
test : 9 2.232522964477539
test_loss : 2.0816636085510254
1801
auto_encoding_loss 0.010268421843647957
commit_loss 0.004042941145598888
zi_prediction_loss 0.9416335821151733
1801 0.9559449553489685
1802
auto_encoding_loss 0.011044444516301155
commit_loss 0.0042716581374406815
zi_prediction_loss 1.033685564994812
1802 1.049001693725586
1803
auto_encoding_loss 0.009687373414635658
commit_loss 0.004277845844626427
zi_prediction_loss 0.9541640281677246
1803 0.9681292772293091
1804
auto_encoding_loss 0.010270161554217339
commit_loss 0.004372389521449804
zi_prediction_loss 0.9795064330101013
1804 0.9941489696502686
1805
auto_encoding_loss 0.010225108824670315
commit_loss 0.004475221037864685
zi_prediction_loss 1.0122965574264526
1805 1.0269968509